In [33]:
import  pandas as pd
import  os
from    datetime import datetime
from    tqdm import tqdm
import  regex as re
import  time
import  numpy as np
import  random
import  pyperclip as py
# import  PyPDF2
import  openai 
import  regex as re
import  random 
import  os

In [61]:

def calculate_cost_df(df):
    """
    Calculate the total cost of purchasing ingredients for a dish.

    Args:
    df (pd.DataFrame): DataFrame containing dish ingredients and prices.

    Returns:
    pd.DataFrame: DataFrame with an additional column for the total cost of purchasing ingredients for each dish.
    """
    # Calculate the cost for purchasing each ingredient
    # Assuming 'price' is the price for 'purchase_size' amount of the ingredient
    for item in df['ingredients'].unique():
        price = df[df['ingredients'] == item]['price'].values[0]
        amount = df[df['ingredients'] == item]['amounts'].values[0]
        purchase_size = df[df['ingredients'] == item]['purchase_size'].values[0]

        if amount > purchase_size:
            upper = np.ceil(amount / purchase_size)
            purchase_cost = upper * price
        else:
            purchase_cost = price

        df.loc[df['ingredients'] == item, 'price'] = purchase_cost
            
    # Calculate the total cost for each dish
    total_cost = df.groupby('urls')['price'].sum().reset_index()

    # Merge total cost back into the original dataframe
    df = df.merge(total_cost, on='urls', how='left', suffixes=('', '_total'))

    return df

def ingredient_divider(url):
    local_ingredient_amount = len(pantry[pantry.urls == url])
    local_matches = len(pantry[(pantry.urls == url) & (pantry.in_recipe == True)])
    if local_ingredient_amount == 0 or local_matches == 0:
        return 0
    else:
        return local_matches / local_ingredient_amount

def matching_ingredients(url):
    sample_ingredients = pantry[pantry.urls == url].ingredients.tolist()
    pantry['in_recipe'] = False
    pantry.loc[pantry.ingredients.isin(sample_ingredients), 'in_recipe'] = True
    recipes['matches'] = recipes.urls.apply(ingredient_divider)

def matching_ingredient_df(df):
    sample_ingredients = df.ingredients.tolist()
    pantry['in_recipe'] = False
    pantry.loc[pantry.ingredients.isin(sample_ingredients), 'in_recipe'] = True
    recipes['matches'] = recipes.urls.apply(ingredient_divider)

def check_fridge_availability(ingredient, amount_needed):
    """
    Check if the ingredient is available in the fridge and if there is enough of it.

    Args:
    ingredient (str): The name of the ingredient.
    amount_needed (float): The amount of the ingredient needed.

    Returns:
    float: The amount that still needs to be purchased.
    """
    if ingredient in fridge['ingredients'].values:
        available_amount = fridge[fridge['ingredients'] == ingredient]['amounts'].values[0]
        if available_amount >= amount_needed and amount_needed > 0:
            # Enough available, no need to purchase more
            return 0
        else:
            # Some amount still needs to be purchased
            return amount_needed - available_amount
    else:
        # Ingredient not in fridge, need to purchase the entire amount
        return amount_needed

def calculate_cost_and_remaining(row):
    amount_to_purchase = check_fridge_availability(row['ingredients'], row['amounts'])
    if row['purchase_size'] == 0 or pd.isna(row['purchase_size']):
        # Handle the case where purchase_size is zero or NaN
        # You can return a default value or handle it as per your requirement
        return pd.Series([0, 0], index=['ingredient_cost', 'remaining'])
    
    if amount_to_purchase > 0:
        if amount_to_purchase > row['purchase_size']:
            num_units = np.ceil(amount_to_purchase / row['purchase_size'])
            cost = num_units * row['local_price']
            remaining = (num_units * row['purchase_size']) - amount_to_purchase
        else:
            cost = row['local_price']
            remaining = row['purchase_size'] - amount_to_purchase
    else:
        cost = 0
        remaining = 0  # Or adjust based on what's left in the fridge
    
    return pd.Series([cost, remaining], index=['ingredient_cost', 'remaining'])

# Existing function
def calculate_cost_of_urls(urls):
    # Assuming combined_df is your DataFrame containing the grocery list
    combined_df = pantry[pantry.urls.isin(urls)].copy()
    # Group by ingredients to sum the amounts needed for each
    grouped_df = combined_df.groupby('ingredients').agg({'amounts': 'sum', 'purchase_size': 'first', 'local_price': 'first'}).reset_index()

    # Apply the function to calculate the cost and remaining amount for each ingredient
    grouped_df[['ingredient_cost', 'remaining']] = grouped_df.apply(calculate_cost_and_remaining, axis=1)

    # Summing up the total cost
    total_cost = grouped_df['ingredient_cost'].sum()

    # print(f"Total Cost: {total_cost}")
    grouped_df['cumsum'] = grouped_df['ingredient_cost'].cumsum()
    return grouped_df


In [79]:
recipes = pd.read_csv('recipes_2.csv')
ingredients_df = pd.read_csv('ingredients_df.csv')
pantry = pd.read_csv('matching.csv')
# lets go to the download folder and import the ingredients_3.csv file
conversion = pd.read_csv('conversion.csv')

fridge = pd.DataFrame(columns=['ingredients', 'amounts'])

In [80]:
ingredients_df

,ingredients,pareto,purchase_unit,type,price,purchase_size,count
0,smør,True,g,0.0,26.0,200.0,577
1,olivenolie,True,ml,0.0,160.0,1000.0,570
2,sukker,True,g,0.0,25.0,1000.0,566
3,hvidløg,True,fed,1.0,30.0,12.0,502
4,hvedemel,True,g,0.0,18.0,2000.0,458
...,...,...,...,...,...,...,...
803,julesalat,False,NaN,NaN,NaN,NaN,1
804,kageplast,False,NaN,NaN,NaN,NaN,1
805,dryp balsamico glace,False,NaN,NaN,NaN,NaN,1
806,dryp honning,False,NaN,NaN,NaN,NaN,1


In [35]:
df = pd.read_csv('similiarity.csv', index_col=0)
df

,urls,matches_x,matches_y,https://www.valdemarsro.dk/pasta-med-graeskar/,https://www.valdemarsro.dk/blomkaal-og-halloumi-i-sursoed-sauce/,https://www.valdemarsro.dk/vegetar-taco/,https://www.valdemarsro.dk/bowl-med-spicy-blomkaal/,https://www.valdemarsro.dk/vegetar-ristaffel-med-groentsager/,https://www.valdemarsro.dk/roeget-paprikablomkaal-og-appelsinsalat/,https://www.valdemarsro.dk/hoisin-nudler-med-groent-og-aeg/,...,https://www.valdemarsro.dk/avocadosuppe/,https://www.valdemarsro.dk/dhal/,https://www.valdemarsro.dk/pizza-stromboli/,https://www.valdemarsro.dk/hvid-pizza-med-kartoffel-og-rosmarin/,https://www.valdemarsro.dk/pizza-margherita/,https://www.valdemarsro.dk/alt-godt-fra-koekkenhaven/,https://www.valdemarsro.dk/raita-agurk-og-mynte-sauce/,https://www.valdemarsro.dk/pasta-med-aerter-og-citron/,https://www.valdemarsro.dk/blomkaalsgratin/,https://www.valdemarsro.dk/i-morgen-udkommer-min-nye-kogebog/
0,https://www.valdemarsro.dk/indisk-vegetar-bowl/,1.000000,1.000000,0.130435,0.260870,0.260870,0.478261,0.347826,0.130435,0.173913,...,0.173913,0.304348,0.130435,0.043478,0.130435,0.217391,0.130435,0.043478,0.130435,0
1,https://www.valdemarsro.dk/pasta-med-graeskar/,0.272727,0.272727,1.000000,0.181818,0.090909,0.181818,0.181818,0.090909,0.181818,...,0.181818,0.181818,0.181818,0.090909,0.181818,0.181818,0.000000,0.181818,0.181818,0
2,https://www.valdemarsro.dk/blomkaal-og-halloum...,0.375000,0.375000,0.125000,1.000000,0.250000,0.500000,0.312500,0.187500,0.250000,...,0.062500,0.187500,0.125000,0.062500,0.187500,0.125000,0.000000,0.062500,0.062500,0
3,https://www.valdemarsro.dk/vegetar-taco/,0.375000,0.375000,0.062500,0.250000,1.000000,0.500000,0.125000,0.375000,0.250000,...,0.250000,0.187500,0.062500,0.062500,0.062500,0.187500,0.062500,0.062500,0.062500,0
4,https://www.valdemarsro.dk/bowl-med-spicy-blom...,0.523810,0.523810,0.095238,0.380952,0.380952,1.000000,0.333333,0.238095,0.190476,...,0.142857,0.238095,0.142857,0.142857,0.190476,0.190476,0.047619,0.047619,0.047619,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134,https://www.valdemarsro.dk/alt-godt-fra-koekke...,0.312500,0.312500,0.125000,0.125000,0.187500,0.250000,0.312500,0.125000,0.187500,...,0.062500,0.250000,0.125000,0.125000,0.187500,1.000000,0.125000,0.062500,0.000000,0
135,https://www.valdemarsro.dk/raita-agurk-og-mynt...,0.600000,0.600000,0.000000,0.000000,0.200000,0.200000,0.200000,0.000000,0.000000,...,0.200000,0.800000,0.200000,0.000000,0.000000,0.400000,1.000000,0.000000,0.000000,0
136,https://www.valdemarsro.dk/pasta-med-aerter-og...,0.200000,0.200000,0.400000,0.200000,0.200000,0.200000,0.200000,0.200000,0.200000,...,0.000000,0.200000,0.200000,0.200000,0.200000,0.200000,0.000000,1.000000,0.000000,0
137,https://www.valdemarsro.dk/blomkaalsgratin/,0.375000,0.375000,0.250000,0.125000,0.125000,0.125000,0.125000,0.125000,0.125000,...,0.000000,0.000000,0.125000,0.125000,0.000000,0.000000,0.000000,0.000000,1.000000,0


In [36]:
df.drop(columns=['matches_x', 'matches_y'], inplace=True)

,urls,https://www.valdemarsro.dk/pasta-med-graeskar/,https://www.valdemarsro.dk/blomkaal-og-halloumi-i-sursoed-sauce/,https://www.valdemarsro.dk/vegetar-taco/,https://www.valdemarsro.dk/bowl-med-spicy-blomkaal/,https://www.valdemarsro.dk/vegetar-ristaffel-med-groentsager/,https://www.valdemarsro.dk/roeget-paprikablomkaal-og-appelsinsalat/,https://www.valdemarsro.dk/hoisin-nudler-med-groent-og-aeg/,https://www.valdemarsro.dk/okonomiyaki/,https://www.valdemarsro.dk/samosa/,...,https://www.valdemarsro.dk/avocadosuppe/,https://www.valdemarsro.dk/dhal/,https://www.valdemarsro.dk/pizza-stromboli/,https://www.valdemarsro.dk/hvid-pizza-med-kartoffel-og-rosmarin/,https://www.valdemarsro.dk/pizza-margherita/,https://www.valdemarsro.dk/alt-godt-fra-koekkenhaven/,https://www.valdemarsro.dk/raita-agurk-og-mynte-sauce/,https://www.valdemarsro.dk/pasta-med-aerter-og-citron/,https://www.valdemarsro.dk/blomkaalsgratin/,https://www.valdemarsro.dk/i-morgen-udkommer-min-nye-kogebog/
0,https://www.valdemarsro.dk/indisk-vegetar-bowl/,0.130435,0.260870,0.260870,0.478261,0.347826,0.130435,0.173913,0.260870,0.391304,...,0.173913,0.304348,0.130435,0.043478,0.130435,0.217391,0.130435,0.043478,0.130435,0
1,https://www.valdemarsro.dk/pasta-med-graeskar/,1.000000,0.181818,0.090909,0.181818,0.181818,0.090909,0.181818,0.181818,0.181818,...,0.181818,0.181818,0.181818,0.090909,0.181818,0.181818,0.000000,0.181818,0.181818,0
2,https://www.valdemarsro.dk/blomkaal-og-halloum...,0.125000,1.000000,0.250000,0.500000,0.312500,0.187500,0.250000,0.250000,0.312500,...,0.062500,0.187500,0.125000,0.062500,0.187500,0.125000,0.000000,0.062500,0.062500,0
3,https://www.valdemarsro.dk/vegetar-taco/,0.062500,0.250000,1.000000,0.500000,0.125000,0.375000,0.250000,0.125000,0.250000,...,0.250000,0.187500,0.062500,0.062500,0.062500,0.187500,0.062500,0.062500,0.062500,0
4,https://www.valdemarsro.dk/bowl-med-spicy-blom...,0.095238,0.380952,0.380952,1.000000,0.333333,0.238095,0.190476,0.285714,0.333333,...,0.142857,0.238095,0.142857,0.142857,0.190476,0.190476,0.047619,0.047619,0.047619,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134,https://www.valdemarsro.dk/alt-godt-fra-koekke...,0.125000,0.125000,0.187500,0.250000,0.312500,0.125000,0.187500,0.125000,0.250000,...,0.062500,0.250000,0.125000,0.125000,0.187500,1.000000,0.125000,0.062500,0.000000,0
135,https://www.valdemarsro.dk/raita-agurk-og-mynt...,0.000000,0.000000,0.200000,0.200000,0.200000,0.000000,0.000000,0.000000,0.600000,...,0.200000,0.800000,0.200000,0.000000,0.000000,0.400000,1.000000,0.000000,0.000000,0
136,https://www.valdemarsro.dk/pasta-med-aerter-og...,0.400000,0.200000,0.200000,0.200000,0.200000,0.200000,0.200000,0.200000,0.200000,...,0.000000,0.200000,0.200000,0.200000,0.200000,0.200000,0.000000,1.000000,0.000000,0
137,https://www.valdemarsro.dk/blomkaalsgratin/,0.250000,0.125000,0.125000,0.125000,0.125000,0.125000,0.125000,0.250000,0.125000,...,0.000000,0.000000,0.125000,0.125000,0.000000,0.000000,0.000000,0.000000,1.000000,0


In [39]:
df.set_index('urls', inplace=True)
df

,https://www.valdemarsro.dk/pasta-med-graeskar/,https://www.valdemarsro.dk/blomkaal-og-halloumi-i-sursoed-sauce/,https://www.valdemarsro.dk/vegetar-taco/,https://www.valdemarsro.dk/bowl-med-spicy-blomkaal/,https://www.valdemarsro.dk/vegetar-ristaffel-med-groentsager/,https://www.valdemarsro.dk/roeget-paprikablomkaal-og-appelsinsalat/,https://www.valdemarsro.dk/hoisin-nudler-med-groent-og-aeg/,https://www.valdemarsro.dk/okonomiyaki/,https://www.valdemarsro.dk/samosa/,https://www.valdemarsro.dk/blomkaalsdeller/,...,https://www.valdemarsro.dk/avocadosuppe/,https://www.valdemarsro.dk/dhal/,https://www.valdemarsro.dk/pizza-stromboli/,https://www.valdemarsro.dk/hvid-pizza-med-kartoffel-og-rosmarin/,https://www.valdemarsro.dk/pizza-margherita/,https://www.valdemarsro.dk/alt-godt-fra-koekkenhaven/,https://www.valdemarsro.dk/raita-agurk-og-mynte-sauce/,https://www.valdemarsro.dk/pasta-med-aerter-og-citron/,https://www.valdemarsro.dk/blomkaalsgratin/,https://www.valdemarsro.dk/i-morgen-udkommer-min-nye-kogebog/
urls,,,,,,,,,,,,,,,,,,,,,
https://www.valdemarsro.dk/indisk-vegetar-bowl/,0.130435,0.260870,0.260870,0.478261,0.347826,0.130435,0.173913,0.260870,0.391304,0.173913,...,0.173913,0.304348,0.130435,0.043478,0.130435,0.217391,0.130435,0.043478,0.130435,0
https://www.valdemarsro.dk/pasta-med-graeskar/,1.000000,0.181818,0.090909,0.181818,0.181818,0.090909,0.181818,0.181818,0.181818,0.181818,...,0.181818,0.181818,0.181818,0.090909,0.181818,0.181818,0.000000,0.181818,0.181818,0
https://www.valdemarsro.dk/blomkaal-og-halloumi-i-sursoed-sauce/,0.125000,1.000000,0.250000,0.500000,0.312500,0.187500,0.250000,0.250000,0.312500,0.187500,...,0.062500,0.187500,0.125000,0.062500,0.187500,0.125000,0.000000,0.062500,0.062500,0
https://www.valdemarsro.dk/vegetar-taco/,0.062500,0.250000,1.000000,0.500000,0.125000,0.375000,0.250000,0.125000,0.250000,0.125000,...,0.250000,0.187500,0.062500,0.062500,0.062500,0.187500,0.062500,0.062500,0.062500,0
https://www.valdemarsro.dk/bowl-med-spicy-blomkaal/,0.095238,0.380952,0.380952,1.000000,0.333333,0.238095,0.190476,0.285714,0.333333,0.142857,...,0.142857,0.238095,0.142857,0.142857,0.190476,0.190476,0.047619,0.047619,0.047619,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
https://www.valdemarsro.dk/alt-godt-fra-koekkenhaven/,0.125000,0.125000,0.187500,0.250000,0.312500,0.125000,0.187500,0.125000,0.250000,0.125000,...,0.062500,0.250000,0.125000,0.125000,0.187500,1.000000,0.125000,0.062500,0.000000,0
https://www.valdemarsro.dk/raita-agurk-og-mynte-sauce/,0.000000,0.000000,0.200000,0.200000,0.200000,0.000000,0.000000,0.000000,0.600000,0.000000,...,0.200000,0.800000,0.200000,0.000000,0.000000,0.400000,1.000000,0.000000,0.000000,0
https://www.valdemarsro.dk/pasta-med-aerter-og-citron/,0.400000,0.200000,0.200000,0.200000,0.200000,0.200000,0.200000,0.200000,0.200000,0.200000,...,0.000000,0.200000,0.200000,0.200000,0.200000,0.200000,0.000000,1.000000,0.000000,0


In [56]:
# now lets convert it to a df with the columns kn the first column and the index in the second column and the values in the third column
new_df = df.stack().reset_index()
new_df.columns = ['url1', 'url2', 'similarity']
new_df.sort_values(by='similarity', ascending=False, inplace=True)
new_df['name'] = new_df.url1 == new_df.url2
new_df = new_df[new_df.name == False].copy()
new_df = new_df[new_df.similarity > 0.5]
new_df.reset_index(inplace=True, drop=True)
new_df.reset_index(inplace=True)


In [57]:
new_df['index'] = 'recipe '  + new_df['index'].astype(str)
new_df  

,index,url1,url2,similarity,name
0,recipe 0,https://www.valdemarsro.dk/tortellini-i-fad/,https://www.valdemarsro.dk/one-pot-tortellini-...,0.928571,False
1,recipe 1,https://www.valdemarsro.dk/groenkaalssuppe/,https://www.valdemarsro.dk/broccolisuppe/,0.888889,False
2,recipe 2,https://www.valdemarsro.dk/pasta-med-squash/,https://www.valdemarsro.dk/one-pot-pasta-med-s...,0.875000,False
3,recipe 3,https://www.valdemarsro.dk/pasta-med-squash/,https://www.valdemarsro.dk/tortellini-i-fad/,0.875000,False
4,recipe 4,https://www.valdemarsro.dk/hvid-pizza-med-kart...,https://www.valdemarsro.dk/bradepandepizza-med...,0.875000,False
...,...,...,...,...,...
432,recipe 432,https://www.valdemarsro.dk/bowl-med-spicy-blom...,https://www.valdemarsro.dk/asiatisk-vegetarbow...,0.523810,False
433,recipe 433,https://www.valdemarsro.dk/tacosuppe/,https://www.valdemarsro.dk/tex-mex-salat/,0.521739,False
434,recipe 434,https://www.valdemarsro.dk/indisk-vegetar-bowl/,https://www.valdemarsro.dk/harira-marokkansk-s...,0.521739,False
435,recipe 435,https://www.valdemarsro.dk/tacosuppe/,https://www.valdemarsro.dk/vegetar-enchiladas/,0.521739,False


In [71]:
ingredients_df

,ingredients,pareto,purchase_unit,spice,price,purchase_size
0,smør,True,g,0.0,26.0,200.0
1,smør,True,g,0.0,26.0,200.0
2,smør,True,g,0.0,26.0,200.0
3,olivenolie,True,ml,0.0,160.0,1000.0
4,olivenolie,True,ml,0.0,160.0,1000.0
...,...,...,...,...,...,...
1181,rapsolie,False,NaN,NaN,NaN,NaN
1182,kahlua isterninger,False,NaN,NaN,NaN,NaN
1183,rabarbersaft,False,NaN,NaN,NaN,NaN
1184,cm stave,False,NaN,NaN,NaN,NaN


In [90]:
new_df.rename(columns={'index': 'recipe_id'}, inplace=True)

In [91]:
new_df

,recipe_id,url1,url2,similarity,name
0,recipe 0,https://www.valdemarsro.dk/tortellini-i-fad/,https://www.valdemarsro.dk/one-pot-tortellini-...,0.928571,False
1,recipe 1,https://www.valdemarsro.dk/groenkaalssuppe/,https://www.valdemarsro.dk/broccolisuppe/,0.888889,False
2,recipe 2,https://www.valdemarsro.dk/pasta-med-squash/,https://www.valdemarsro.dk/one-pot-pasta-med-s...,0.875000,False
3,recipe 3,https://www.valdemarsro.dk/pasta-med-squash/,https://www.valdemarsro.dk/tortellini-i-fad/,0.875000,False
4,recipe 4,https://www.valdemarsro.dk/hvid-pizza-med-kart...,https://www.valdemarsro.dk/bradepandepizza-med...,0.875000,False
...,...,...,...,...,...
432,recipe 432,https://www.valdemarsro.dk/bowl-med-spicy-blom...,https://www.valdemarsro.dk/asiatisk-vegetarbow...,0.523810,False
433,recipe 433,https://www.valdemarsro.dk/tacosuppe/,https://www.valdemarsro.dk/tex-mex-salat/,0.521739,False
434,recipe 434,https://www.valdemarsro.dk/indisk-vegetar-bowl/,https://www.valdemarsro.dk/harira-marokkansk-s...,0.521739,False
435,recipe 435,https://www.valdemarsro.dk/tacosuppe/,https://www.valdemarsro.dk/vegetar-enchiladas/,0.521739,False


In [94]:
gpt_recipes = pd.DataFrame(columns = ['urls', 'ingredients', 'amounts', 'unit'])

for i in range(len(new_df)):
    row = new_df.iloc[i]
    name = row.recipe_id
    urls = [row.url1, row.url2]
    local_df = calculate_cost_of_urls(urls)
    # now lets merge the ingredients_df with the local_df, so that we can se the units
    local_df = local_df.merge(ingredients_df[['ingredients', 'purchase_unit']], on='ingredients', how='left')
    local_df = local_df[['ingredients', 'remaining', 'purchase_unit']]
    local_df = local_df[local_df.remaining > 0].copy()
    local_df.rename(columns={'remaining': 'amounts'}, inplace=True)

    for i in range(len(local_df)):
        row = local_df.iloc[i]
        new_line = [name, row.ingredients, row.amounts, row.purchase_unit]
        gpt_recipes.loc[len(gpt_recipes)] = new_line

    


In [95]:
gpt_recipes

,urls,ingredients,amounts,unit
0,recipe 0,grøntsagsbouillon,194.00,NaN
1,recipe 0,hvidløg,9.00,fed
2,recipe 0,løg,5.00,stk
3,recipe 0,olivenolie,999.00,ml
4,recipe 0,parmesan,125.00,g
...,...,...,...,...
6322,recipe 436,rød chili,0.25,stk
6323,recipe 436,rød peberfrugt,2.50,stk
6324,recipe 436,safran,0.50,stk
6325,recipe 436,squash,0.50,stk


In [98]:
url = gpt_recipes.urls.sample(1).values[0]
local_df = gpt_recipes[gpt_recipes.urls == url].copy()

def texitify_df(df):
    y = ''
    df = df.reset_index(drop=True)
    for i in range(len(df)):
        row = df.iloc[i]
        y += str(row.amounts) + ' ' + str(row.unit) + ' ' + str(row.ingredients) + '\n'

    return y

y = texitify_df(local_df)
py.copy(y)

In [100]:
gpt_recipes.to_csv('gpt_recipes.csv', index=False)